# Logical Progression ChatBot

In [1]:
# Install Dependencies
!make install

python3 -m pip install --upgrade pip && pip install -q -r requirements.txt


## Load Document

In [2]:
file_path = "docs/LPYear1-10.pdf"
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(file_path)
pages = loader.load()

In [3]:
len(pages)

6870

In [7]:
print(pages[0].page_content[:500])

بسم الله الرحمن الرحٌم  
 
الحمد لله رب العالمٌن  
 
اللهم صلً وسلم وبارك على نبٌنا محمد وعلى صحبه أجمعٌن  
 
اللهُمَّ لا سَهْلَ إلا مَا جَعَلتَهُ سَهْلا وَ أنتَ تَجْعَلُ الحزْنَ إذا شِئْتَ سَهْلا 
 
اللهم أعنا على ذكرك وشكرك وحسن عبادتك ٌا رب الكرٌم  
 
We begin with the name of Allāh (sub ḥānahu wa taʿāla),  we praise Him , and we hope that He will help 
us remember Him, we hope that He will help us thank Him --because we’re pretty bad at that, to be 
honest , when left to our own devices --an


## Split document

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [9]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [ ]:
docs = text_splitter.split_documents(pages)

In [14]:
print(docs[0].page_content[-200:])

cited in the prayer,  after the prayer as well, and in 
general, when one feels the need.  
 
I ask Allāh (sub ḥānahu wa taʿāla)  to make this easy. There is no ease, and nothing happens except by His


In [15]:
rec_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
rdocs = rec_splitter.split_documents(pages)
len(rdocs)

19856

In [17]:
print(rdocs[0].page_content[-200:])

cite from the Qur ʾān so that we can hopefully start this journey  with so me 
barakah, and some ayā t which I think are very pertinent to the reason why we’re here, what the goa l of 
the journey is.


# Embeddings

In [18]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/multilingual-e5-large")

/Users/iolanigan/LangChain/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
from langchain.vectorstores import Chroma
persist_directory = 'db/chroma/'
!rm -rf ./db/chroma

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings

# model_name = "sentence-transformers/all-mpnet-base-v2"
model_name = "intfloat/multilingual-e5-large"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [26]:
vectorstore = Chroma("lp_store", embedding)
# vectordb = Chroma.from_documents(
#     documents=rdocs,
#     embedding=embedding,
#     persist_directory=persist_directory
# )

In [ ]:
vectorstore.add_documents(rdocs)